# Data Fetching

In [1]:
import requests
import pandas as pd
import json
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
CLIENT_ID = 'e1da2be681094c9099d7df7ccccbfdba'
CLIENT_SECRET = 'b85a78fe8d2c41f688b7455303beb7c4'

In [3]:
df = pd.read_csv('/content/drive/My Drive/CS514/Project/List of most-streamed songs on Spotify.csv')

top100song_artist_list = df['Artist'].to_list()
top100song_artist_list = [e.lower() for e in top100song_artist_list]
print(len(top100song_artist_list))
top100song = df['Song'].to_list()
listen_count = df['Streams (Billions)'].apply(lambda x: float(x.replace(',','.'))).to_list()

def fill_dict(artist_dict, first_artist, second_artist):
  if(first_artist not in artist_dict):
    artist_info_dict = dict()
    artist_dict[first_artist] = artist_info_dict

    artist_dict[first_artist]['Songs'] = dict()
    artist_dict[first_artist]['Songs'][top100song[i]] = listen_count[i]

    artist_dict[first_artist]['Feat'] = dict()
  else:
    artist_dict[first_artist]['Songs'][top100song[i]] = listen_count[i]
  if(second_artist not in artist_dict):
    artist_info_dict = dict()
    artist_dict[second_artist] = artist_info_dict

    artist_dict[second_artist]['Songs'] = dict()
    artist_dict[second_artist]['Songs'][top100song[i]] = listen_count[i]

    artist_dict[second_artist]['Feat'] = dict()
  else:
    artist_dict[second_artist]['Songs'][top100song[i]] = listen_count[i]

artist_dict = dict()

for i in range(len(top100song_artist_list)):
  if(not ('and' in top100song_artist_list[i] or 'featuring' in top100song_artist_list[i] or "with" in top100song_artist_list[i])  or  
     ('and' in top100song_artist_list[i] and (len(top100song_artist_list[i][top100song_artist_list[i].find('and')+4: ]) < 2 ))):
    if(top100song_artist_list[i] not in artist_dict):
      artist_info_dict = dict()
      artist_dict[top100song_artist_list[i]] = artist_info_dict

      artist_dict[top100song_artist_list[i]]['Songs'] = dict()
      artist_dict[top100song_artist_list[i]]['Songs'][top100song[i]] = listen_count[i]

      artist_dict[top100song_artist_list[i]]['Feat'] = dict()
    else:
      artist_dict[top100song_artist_list[i]]['Songs'][top100song[i]] = listen_count[i]
  else:
    if('featuring' in top100song_artist_list[i]):
      first_artist = top100song_artist_list[i][:top100song_artist_list[i].find('featuring')-1]
      second_artist = top100song_artist_list[i][top100song_artist_list[i].find('featuring')+10: ]

      fill_dict(artist_dict, first_artist, second_artist)
    elif("with" in top100song_artist_list[i]):
      first_artist = top100song_artist_list[i][:top100song_artist_list[i].find('with')-1]
      second_artist = top100song_artist_list[i][top100song_artist_list[i].find('with')+5: ]

      fill_dict(artist_dict, first_artist, second_artist)
    else:
      #and
      first_artist = top100song_artist_list[i][:top100song_artist_list[i].find('and')-1]
      second_artist = top100song_artist_list[i][top100song_artist_list[i].find('and')+4: ]

      fill_dict(artist_dict, first_artist, second_artist)

100


In [4]:
def get_auth():
  # Set up authentication
  client_id = CLIENT_ID
  client_secret = CLIENT_SECRET
  auth_url = "https://accounts.spotify.com/api/token"
  

  # Get access token
  auth_response = requests.post(auth_url, {
      "grant_type": "client_credentials",
      "client_id": client_id,
      "client_secret": client_secret
  })
  auth_response_data = auth_response.json()
  access_token = auth_response_data["access_token"]
  return access_token

In [5]:

# You can get the access token at first to use it in your requests

access_token = get_auth()

def get_search_res(song_name="", artist="",type="artist", access_token=get_auth()):
  '''
  This function takes a song_name and returns songs with given name
  It is adjusted to bring the most relevant result
  '''
  

  search_url = "https://api.spotify.com/v1/search"
  # Make a request to search for songs
  headers = {
      "Authorization": f"Bearer {access_token}"
  }
  query = song_name + ' ' + artist
  params = {
      "q": query,
      "type": type
  }
  response = requests.get(search_url, headers=headers, params=params)
  # print(response.headers)
  response_data = response.json()
  ret = []
  ret_val={}
  # Extract song or artist information
  if params['type'] == "track":
    tracks = response_data["tracks"]['items']
    for track in tracks:
   
      ret_val = {}
      ret_val["popularity"] = track['popularity']
      ret_val["song_name"] = track["name"]
      ret_val["release_date"] = track["album"]["release_date"]
      ret_val["artists"] = [artist["name"] for artist in track["artists"]]
      ret.append(ret_val)


  elif params["type"] == "artist":
    artists = response_data["artists"]['items']
    for artist in artists:
      ret_val["name"] = artist["name"]
      ret_val["id"] = artist["id"]
      ret_val["popularity"] = artist["popularity"]
      ret_val["followers"] = artist["followers"]["total"]
      ret_val["genres"] = artist["genres"]
      ret_val["collaborations"] = []
      return ret_val
  return ret

  

In [ ]:
print(get_search_res( artist='Ed Sheeran Eminem',access_token=access_token,type="track"))

{'content-type': 'application/json; charset=utf-8', 'cache-control': 'public, max-age=7200', 'x-robots-tag': 'noindex, nofollow', 'access-control-allow-origin': '*', 'access-control-allow-headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token', 'access-control-allow-methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'access-control-allow-credentials': 'true', 'access-control-max-age': '604800', 'content-encoding': 'gzip', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff', 'date': 'Mon, 05 Jun 2023 15:14:45 GMT', 'server': 'envoy', 'Via': 'HTTP/2 edgeproxy, 1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}
[{'popularity': 75, 'song_name': 'River (feat. Ed Sheeran)', 'release_date': '2017-12-15', 'artists': ['Eminem', 'Ed Sheeran']}, {'popularity': 67, 'song_name': 'Those Kinda Nights (feat. 

In [6]:
def get_tracks_of_artist(artist_id,access_token=get_auth()):
  artist_url = "https://api.spotify.com/v1/artists/{}/albums".format('6eUKZXaKkcviH0Ku9w2n3V')  # Replace {artist_id} with the artist's ID

  # Make a request to get the artist's top tracks
  headers = {
      "Authorization": f"Bearer {access_token}"
  }
  params = {
      "country": "US",  # Replace with the desired country code
  }
  response = requests.get(artist_url, headers=headers, params=params)
  response_data = response.json()
  print(response_data)
  # Extract track information
  tracks = response_data["albums"]
  for track in tracks:
 
    print(track['name'])
    print(track['id'])
    print(track['popularity'])
    print(track['album']["release_date"])


In [ ]:
get_tracks_of_artist('6eUKZXaKkcviH0Ku9w2n3V',access_token)

In [7]:
def get_artist_info(artist_id,access_token=get_auth()):
  '''
    Get the artist information when the artist id is provided
  '''
  artist_url = "https://api.spotify.com/v1/artists/{}".format(artist_id)  # Replace {artist_id} with the artist's ID

  # Make a request to get the artist's information
  headers = {
      "Authorization": f"Bearer {access_token}"
  }
  response = requests.get(artist_url, headers=headers)
  response_data = response.json()
  try:
    artist = {}
    # Extract artist information
    artist['name'] = response_data["name"]
    artist['followers'] = response_data["followers"]["total"]
    artist['genres'] = response_data["genres"]
    artist['popularity'] = response_data["popularity"]
    artist['id'] = response_data["id"]
    artist['collaborations'] = []
    return artist
  except:
      print("Invalid id")
      return False

In [ ]:
print(get_artist_info("ASDASDASD"))

Invalid id
False


In [ ]:
'''
Generate a list for artists info then dump the gathered data into a json file to get rid of redundant api calls
'''


top_artist_id_list = list()
artists_info_list = list()

for key in artist_dict.keys():
  artist_info = get_search_res(artist=key,access_token=access_token)
  top_artist_id_list.append(artist_info["id"])
  artists_info_list.append(artist_info)


with open("sample.json", "w") as outfile:
    json.dump(artists_info_list, outfile)

In [8]:
top_artist_id_list = list()
for key in artist_dict.keys():
  artist_info = get_search_res(artist=key,access_token=access_token)
  top_artist_id_list.append(artist_info["id"])


artists_info_list = list()
# Opening JSON file
f = open('/content/drive/My Drive/CS514/Project/sample.json',)
   
# returns JSON object as 
# a dictionary
artists_info_list = json.load(f)
   

In [10]:
top_artist_id_list

['6eUKZXaKkcviH0Ku9w2n3V',
 '1Xyo4u8uXC1ZmMpatF05PJ',
 '2NjfBq1NflQcKSeiDooVjY',
 '4GNC7GD6oZMSxPGyXy4MNB',
 '246dkjvS1zLTtiykXe5h60',
 '1URnnhqYAYcrqrcwql10ft',
 '1zNqQNIdeOUZHb8zbZRFMX',
 '3TVXtAsR1Inumwj472S9r4',
 '3tVQdUvClmAT7URs9V3rsp',
 '69GGBxA162lTqCwzJG5jLp',
 '26VFTg2z8YR0cCuwLzESi2',
 '53XhwfbYqKCa1cC15pYq2q',
 '7n2wHs1TKAczGzO7Dd2rGr',
 '4nDoRrQiYLoBzwC5BhVJzF',
 '2tIP7SsRs7vjIcLrU85W8J',
 '1uNFoZAHBGtllmzznpCI3s',
 '4IWBUUAFIplrNtaOHcJPRM',
 '4yvcSjfu4PC0CYQyLy4wSq',
 '6qqNVTkY8uBg9cP3Jd7DAH',
 '4MCBfE4596Uoi2O4DtmEMz',
 '4tZwfgrHOc3mvqYlEYSvVi',
 '6M2wZ9GZgrQXHCFfjv46we',
 '6LuN9FCkKOj5PcnpouEgny',
 '6KImCVD70vtIoJWnq6nGn3',
 '1dfeR4HaWDbWqFHLkxsg1d',
 '4gzpq5DPGxSnKTe4SA8HAU',
 '1HY2Jd0NmPuamShAr6KMms',
 '4VIvfOurcf0vuLRxLkGnIG',
 '15UsOTVnJzReFVN1VCnxy4',
 '2FXC3k01G6Gw61bmprjgqS',
 '5y2Xq6xcjJb2jVM54GHK3t',
 '77SW9BnxLY8rJ0RciFqkHh',
 '66CXWjxzNUsdJxJ2JdwvnR',
 '0Y5tJX1MQlPlqiwlOH1tJY',
 '2YZyLoL8N0Wb9xBt1NhZWg',
 '50co4Is1HCEo8bhOyUWKpn',
 '4O15NlyKLIASxsJ0PrXPfz',
 

In [ ]:
from datetime import datetime


artists_file = open('sample.json')
artists = json.load(artists_file)

def conv_date(date_str):
  '''This function takes an date string in %Y-%m-%d format 
  and converts that into a date object
  - date_str @string '''
  if(len(date_str) == 4):
    date_str += "-1-1"
  if(len(date_str) == 7):
    date_str += "-1"
  date_object = datetime.strptime(date_str, '%Y-%m-%d').date()
  return date_object


def clear_duplicate(lst):
  '''This function takes the list of albums and clears out duplicate enteries '''
  temp = list()
  for item in lst:
      flag = True
      for item2 in temp:
        if item2["artists"][0] == item["artists"][0] and conv_date(item2["release_date"]) < conv_date(item["release_date"]):
          flag = False
          continue
      if(flag):
        temp.append(item)
  return temp

def get_albums(artists):
  for artist in artists:
    try:
      temp_lst = get_search_res(artist=artist["name"],type="track")
      temp_lst = clear_duplicate(temp_lst)
      artist["albums"] = temp_lst
    except:
      print("An error occured during getting albums")
  return artists

get_albums(artists)

[{'name': 'Ed Sheeran',
  'id': '6eUKZXaKkcviH0Ku9w2n3V',
  'popularity': 91,
  'followers': 112377639,
  'genres': ['pop', 'uk pop'],
  'collaborations': [],
  'albums': [{'popularity': 88,
    'song_name': 'Shivers',
    'release_date': '2021-10-29',
    'artists': ['Ed Sheeran']},
   {'popularity': 87,
    'song_name': 'Bad Habits',
    'release_date': '2021-10-29',
    'artists': ['Ed Sheeran']},
   {'popularity': 90,
    'song_name': 'Perfect',
    'release_date': '2017-03-03',
    'artists': ['Ed Sheeran']},
   {'popularity': 89,
    'song_name': 'Shape of You',
    'release_date': '2017-03-03',
    'artists': ['Ed Sheeran']},
   {'popularity': 81,
    'song_name': 'Castle on the Hill',
    'release_date': '2017-03-03',
    'artists': ['Ed Sheeran']},
   {'popularity': 76,
    'song_name': 'Dive',
    'release_date': '2017-03-03',
    'artists': ['Ed Sheeran']},
   {'popularity': 67,
    'song_name': 'Nancy Mulligan',
    'release_date': '2017-03-03',
    'artists': ['Ed Sheeran'

In [ ]:

with open("artists_complete.json", "w") as outfile:
    json.dump(artists, outfile)

# Popularity Normalization based on Collaboration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
URI = '/content/drive/My Drive/CS514/Project/artists_complete.json'

In [ ]:
with open(URI) as f:
    data = json.load(f)

In [ ]:
df = pd.DataFrame(data)
df_copied = df.copy()

In [ ]:
df.describe()

,popularity,followers
count,14804.000000,1.480400e+04
mean,34.990273,5.692408e+05
std,22.063958,3.153044e+06
min,0.000000,0.000000e+00
25%,17.000000,4.225000e+02
50%,36.000000,8.987000e+03
75%,52.000000,1.322522e+05
max,100.000000,1.119568e+08


In [ ]:
df.head(10)

,name,id,popularity,followers,genres,collaborations,albums
0,Ed Sheeran,6eUKZXaKkcviH0Ku9w2n3V,92,111956794,"[pop, uk pop]",[],"[{'popularity': 88, 'song_name': 'Shivers', 'r..."
1,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,96,64694323,"[canadian contemporary r&b, canadian pop, pop]",[],"[{'popularity': 96, 'song_name': 'Creepin' (wi..."
2,Tones And I,2NjfBq1NflQcKSeiDooVjY,71,3011530,[australian pop],[],"[{'popularity': 79, 'song_name': 'Dance Monkey..."
3,Lewis Capaldi,4GNC7GD6oZMSxPGyXy4MNB,83,11021958,"[pop, uk pop]",[],"[{'popularity': 89, 'song_name': 'Someone You ..."
4,Post Malone,246dkjvS1zLTtiykXe5h60,89,40583301,"[dfw rap, melodic rap, pop, rap]",[],"[{'popularity': 90, 'song_name': 'Chemical', '..."
5,21 Savage,1URnnhqYAYcrqrcwql10ft,91,13648126,"[atl hip hop, hip hop, rap, trap]",[],"[{'popularity': 90, 'song_name': 'Rich Flex', ..."
6,Swae Lee,1zNqQNIdeOUZHb8zbZRFMX,78,1669740,"[melodic rap, rap, trap]",[],"[{'popularity': 84, 'song_name': 'Sunflower - ..."
7,Drake,3TVXtAsR1Inumwj472S9r4,96,75150521,"[canadian hip hop, canadian pop, hip hop, rap,...",[],"[{'popularity': 91, 'song_name': 'Search & Res..."
8,Wizkid,3tVQdUvClmAT7URs9V3rsp,77,3724485,"[afrobeats, afropop, azonto, azontobeats, nige...",[],"[{'popularity': 88, 'song_name': 'One Dance', ..."
9,The Chainsmokers,69GGBxA162lTqCwzJG5jLp,80,20131219,"[electropop, pop, tropical house]",[],"[{'popularity': 86, 'song_name': 'Closer', 're..."


In [10]:
df_feat = pd.read_csv('/content/drive/My Drive/CS514/Project/edges.csv')

In [11]:
def create_collab_array(df_feat,top_artist_id_list):
  '''This fucntion takes the main dataframe with top_artist, then creates a df with first and second degree collabs'''

  mask = df_feat.isin(top_artist_id_list).any(axis=1)

  # Get the collaborations related to top artist first

  first_degree_collab = df_feat.loc[mask]
  print(first_degree_collab.shape)
  # By using a similar mask method, get the second degree collaborations
  secondary_main_artists = first_degree_collab["id_1"].values.tolist()  
  mask2 = df_feat.isin(secondary_main_artists).any(axis=1)

  second_degree_collab =  df_feat.loc[mask2]
  
  # Then concatanate two dataframes which contains first and second degree collaborations 
  all_collabs = pd.concat([first_degree_collab,second_degree_collab])
  print(second_degree_collab.shape) 
  print(second_degree_collab.head)
  return  first_degree_collab, second_degree_collab



first_degree_collabs ,second_degree_collabs = create_collab_array(df_feat,top_artist_id_list)

(6151, 2)
(53025, 2)
<bound method NDFrame.head of                           id_0                    id_1
0       76M2Ekj8bG8W7X2nbx2CpF  7sfl4Xt5KmfyDs2T3SVSMK
3       6PvcxssrQ0QaJVaBWHD07l  6UCQYrcJ6wab6gnQ89OJFh
7       21ey6s5hEcTcVKDrF5MWby  7uMh23xWiuR7zsNkuNcm2G
18      2oX42qP5ineK3hrhBECLmj  6aZyMrc4doVtZyKNilOmwu
26      2cPqdH7XMvwaBJEVjheH8g  329e4yvIujISKGKz1BZZbO
...                        ...                     ...
300335  76fuWYgIf3TVIopTs3vaJ6  7HCqGPJcQTyGJ2yqntbuyr
300344  4abEgIF5hnsotmbZleI4Du  5he5w2lnU9x7JFhnwcekXX
300351  1V3VTM7VspiQjcmRhC010n  6cEuCEZu7PAE9ZSzLLc2oQ
300353  3dz0NnIZhtKKeXZxLOxCam  4AGwPDdh1y8hochNzHy5HC
300385  25uiPmTg16RbhZWAqwLBy5  3Xt3RrJMFv5SZkCfUE8C1J

[53025 rows x 2 columns]>


In [ ]:
first_degree_collabs.head()

In [15]:
first_neigh , sec_neigh = create_collab_array(df_feat,[first_degree_collabs.iloc[0][0]])

(110, 2)
(9123, 2)
<bound method NDFrame.head of                           id_0                    id_1
54      4DdkRBBYG6Yk9Ka8tdJ9BW  5yPzzu25VzEk8qrGTLIrE1
68      0VRj0yCOv2FXJNP47XQnx5  64M6ah0SkkRsnPGtGiRAbb
101     5pUo3fmmHT8bhCyHE52hA6  6rXMet89uui55tWrcEz7ma
111     55Aa2cqylxrFIXC767Z865  6LuN9FCkKOj5PcnpouEgny
119     1RyvyyTE3xzB2ZywiAwp0i  2BF8OuAjYqNqTwSHfFLov8
...                        ...                     ...
300240  4D75GcNG95ebPtNvoNVXhz  7tjVFCxJdwT4NdrTmjyjQ6
300250  1XkoF8ryArs86LZvFOkbyr  1jLOGxY81S3FNq29X3qvmj
300253  1URnnhqYAYcrqrcwql10ft  5AQVXVBupwWBMUC1A2JOkk
300272  47z7ZrgFoBvVpCnElCE3Zh  4ec8qSL8tNFr1heJFmYfux
300273  17lzZA2AlOHwCwFALHttmp  1l7ZsJRRS8wlW3WfJfPfNS

[9123 rows x 2 columns]>


In [ ]:
first_neigh.head()

,id_0,id_1
68,0VRj0yCOv2FXJNP47XQnx5,64M6ah0SkkRsnPGtGiRAbb
2430,0VRj0yCOv2FXJNP47XQnx5,316cPrVpf18LLtyZm7fn3y
10630,0VRj0yCOv2FXJNP47XQnx5,6vXTefBL93Dj5IqAWq6OTv
12103,0VRj0yCOv2FXJNP47XQnx5,3hcs9uc56yIGFCSy9leWe7
12860,0VRj0yCOv2FXJNP47XQnx5,6tQmUQ1aBbgQwEqXFjVwB6


# Colaborations

In [12]:
EDGE_URI = '/content/drive/My Drive/CS514/Project/edges.csv'

In [13]:
edges = pd.read_csv(EDGE_URI)

In [14]:
def get_artist_info_colab(artist_id,  access_token=get_auth()):
  '''
    Get the artist information when the artist id is provided
  '''
  artist_url = "https://api.spotify.com/v1/artists/{}".format(artist_id)  # Replace {artist_id} with the artist's ID
  # Make a request to get the artist's information
  headers = {
      "Authorization": f"Bearer {access_token}"
  }
  response = requests.get(artist_url, headers=headers)
  response_data = response.json()
  try:
    artist = {}
    # Extract artist information
    artist['name'] = response_data["name"]
    artist['followers'] = response_data["followers"]["total"]
    artist['genres'] = response_data["genres"]
    artist['popularity'] = response_data["popularity"]
    artist['id'] = response_data["id"]
    artist['song'] = get_search_res( artist='{}'.format(artist['name']),access_token=access_token,type="track")[0]
    return artist
  except:
      print("Invalid id")
      print(response.headers)
      return False

In [ ]:
get_artist_info_colab("5AQVXVBupwWBMUC1A2JOkk")

Invalid id


False

In [ ]:
def mask(edges, id_list):
  mask = edges.isin(id_list).any(axis=1)
  masked_df = edges.loc[mask]
  return masked_df

In [15]:
def expand_edges(edges_df):
  dict = {"id_0":[],"name_0":[],"id_1":[],"name_1":[],"song" :[]}
  temp_df = pd.DataFrame(dict)
  count = 0
  for ind,row in edges_df.iterrows():
    count += 1
    if(count > 6000):
      print("Terminated manually")
      break
    
    try:
      first_singer = get_artist_info_colab(row["id_0"])
      sec_singer = get_artist_info_colab(row["id_1"])
      song = get_search_res(type="track",artist=(first_singer["name"] + " " + sec_singer["name"]))[0]

      temp_df.loc[len(temp_df.index)] = [first_singer["id"], first_singer["name"],sec_singer["id"], sec_singer["name"],song] 
    except Exception as e:
      print("While getting song by ", row["id_0"],"and",row["id_1"])
      print("Error occured:",e)
  return temp_df

In [16]:
import numpy as np
divided_dfs = np.array_split(second_degree_collabs,10)

In [17]:
temp_df = expand_edges(divided_dfs[4])
temp_df.to_csv("/content/drive/My Drive/CS514/Project/sec_degree_expanded5.csv")

Görüntülenen çıkış son 5000 satıra kısaltıldı.
While getting song by  3GUzDIvG3Rk7T5I4h7SPA0 and 63X1WKthLQidtqxxO2sgeq
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  23zg3TcAtWQy7J6upgbUnj and 64ovxBmjJ9ZDqkzEXVWpVc
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  0EeQBlQJFiAfJeVN2vT9s0 and 54QfCw9UFq4SUZydhuDVSy
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  4BoRxUdrcgbbq1rxJvvhg9 and 6I3M904Y9IwgDjrQ9pANiB
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  3aQeKQSyrW4qWr35idm0cy and 59wfkuBoNyhDMQGCljbUbA
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  0cGUm45nv7Z6M6qdXYQGTX and 43BxCL6t4c73BQnIJtry5v
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  2orIEMw3OyKWIaZ6CGueGg and 4aKr2HYGiJEtfIDk52N0eL
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  1P8IfcNKwrkQP5xJWuha

In [18]:
temp_df = expand_edges(divided_dfs[5])
temp_df.to_csv("/content/drive/My Drive/CS514/Project/sec_degree_expanded6.csv")

While getting song by  1qUjOF5fzrpoNycD36b2jZ and 78xUyw6FkVZrRAtziFdtdu
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  2zx8hW1HdhkTCMa6V2GGNa and 6jQZzWW3JlEtcRDSzFVKSP
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  77AiFEVeAVj2ORpC85QVJs and 7tm9Tuc70geXOOyKhtZHIj
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  4I22x3YxEveEfbtVDj54H3 and 7dc6hUwyuIhrZdh80eaCEE
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  2MzzwDQ7n8wSGCJIqKJH5r and 66CXWjxzNUsdJxJ2JdwvnR
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  3GcHy1JxyiaqMpImZQ3FKl and 7vb7VLDqpLTlAy1ctTMR5d
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  3RtNN1VnooWEn3KQk03DUL and 6w3SkAHYPsQ1bxV7VDlG5y
Error occured: Expecting value: line 1 column 1 (char 0)
While getting song by  1X0cZTSRuf2ph6dTVs8J5k and 41X1TR6hrK8Q2ZCpp2EqCz
Error occured: Ex

KeyboardInterrupt: ignored

In [ ]:
temp_df = expand_edges(divided_dfs[6])
temp_df.to_csv("/content/drive/My Drive/CS514/Project/sec_degree_expanded7.csv")

In [ ]:

temp_df = expand_edges(divided_dfs[7])
temp_df.to_csv("/content/drive/My Drive/CS514/Project/sec_degree_expanded8.csv")

In [ ]:
temp_df.to_csv("/content/drive/My Drive/CS514/Project/edges_expanded3.csv")

In [ ]:
def collaborations(df, edges):

  edges = mask(edges, df['id'].to_list())

  for index, row in edges.iterrows():
    id_0 = row['id_0']
    id_1 = row['id_1']
    
    if(id_0 in df['id'].values):
      
      artist_name = df.loc[df['id'] == id_0, 'name'].values[0]
      artist = get_artist_info_colab(id_1, artist_name)

      df.loc[df['id'] == id_0, 'collaborations'].values[0].append(artist)

    if(id_1 in df['id'].values):
      
      artist_name = df.loc[df['id'] == id_1, 'name'].values[0]
      artist = get_artist_info_colab(id_0, artist_name)

      df.loc[df['id'] == id_1, 'collaborations'].values[0].append(artist)


  # collaborations = df.loc[df['id'] == '6eUKZXaKkcviH0Ku9w2n3V', 'collaborations'].values[0]
  


  return df

In [ ]:
df = collaborations(df.head(10), edges)

In [ ]:
df.head(10)

,name,id,popularity,followers,genres,collaborations,albums
0,Ed Sheeran,6eUKZXaKkcviH0Ku9w2n3V,92,111956794,"[pop, uk pop]","[{'name': 'Ghali', 'followers': 2956063, 'genr...","[{'popularity': 88, 'song_name': 'Shivers', 'r..."
1,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,96,64694323,"[canadian contemporary r&b, canadian pop, pop]","[{'name': 'Young Thug', 'followers': 8349449, ...","[{'popularity': 96, 'song_name': 'Creepin' (wi..."
2,Tones And I,2NjfBq1NflQcKSeiDooVjY,71,3011530,[australian pop],"[{'name': 'Macklemore', 'followers': 2066830, ...","[{'popularity': 79, 'song_name': 'Dance Monkey..."
3,Lewis Capaldi,4GNC7GD6oZMSxPGyXy4MNB,83,11021958,"[pop, uk pop]","[{'name': 'Future Humans', 'followers': 2760, ...","[{'popularity': 89, 'song_name': 'Someone You ..."
4,Post Malone,246dkjvS1zLTtiykXe5h60,89,40583301,"[dfw rap, melodic rap, pop, rap]","[{'name': 'Nicki Minaj', 'followers': 28264860...","[{'popularity': 90, 'song_name': 'Chemical', '..."
5,21 Savage,1URnnhqYAYcrqrcwql10ft,91,13648126,"[atl hip hop, hip hop, rap, trap]","[{'name': 'Miguel', 'followers': 4645205, 'gen...","[{'popularity': 90, 'song_name': 'Rich Flex', ..."
6,Swae Lee,1zNqQNIdeOUZHb8zbZRFMX,78,1669740,"[melodic rap, rap, trap]","[{'name': 'Shenseea', 'followers': 561222, 'ge...","[{'popularity': 84, 'song_name': 'Sunflower - ..."
7,Drake,3TVXtAsR1Inumwj472S9r4,96,75150521,"[canadian hip hop, canadian pop, hip hop, rap,...","[{'name': 'French Montana', 'followers': 50290...","[{'popularity': 91, 'song_name': 'Search & Res..."
8,Wizkid,3tVQdUvClmAT7URs9V3rsp,77,3724485,"[afrobeats, afropop, azonto, azontobeats, nige...","[{'name': 'AIRIETJA', 'followers': 73, 'genres...","[{'popularity': 88, 'song_name': 'One Dance', ..."
9,The Chainsmokers,69GGBxA162lTqCwzJG5jLp,80,20131219,"[electropop, pop, tropical house]","[{'name': 'neutral.', 'followers': 1929, 'genr...","[{'popularity': 86, 'song_name': 'Closer', 're..."


In [ ]:
# Convert DataFrame to JSON
json_data = df.to_json(orient='records')

# Save JSON data to a file
file_path = '/content/drive/My Drive/Data/CS414 Project/data/sample_collab_data.json'  # Replace with your desired file path
with open(file_path, 'w') as file:
    file.write(json_data)

print("JSON file saved successfully.")

JSON file saved successfully.
